We check the ancilla-free decomposition of a CZ gate obtained via ZX calculus (Table VI in M. Schumann et al "Bridging wire and gate cutting with ZX-calculus" (2025))

In [17]:
import numpy as np
from util import return_ptm

In [18]:
def apply_mcphase_theta(op: np.ndarray, params: dict):
    n = int(np.log2(op.shape[1]))
    if op.shape[1] != 2**n:
        raise ValueError("Matrix shape error: the matrix dimension must be a power of 2")
    theta = params["theta"]
    mcphase = np.identity(2**n, dtype=complex)
    mcphase[-1, -1] = np.exp(1j*theta)
    return mcphase@op@mcphase.conj().T 

In [19]:
num_qubits = 2
m = 1
m_prime = num_qubits - m

In [20]:
target_ptm_cz = return_ptm(apply_mcphase_theta, num_qubits, params={"theta": np.pi})

Now we obtain the PTMs for each term in the decomposition
1. $\mathcal{R}_Z(\pi/2) \otimes \mathcal{R}_Z(\pi/2)$

In [21]:
deco_dict = {}
deco_dict["1"] = {}
deco_dict["1"]["q"] = 1/2
deco_dict["1"]["ptm"] = np.kron(return_ptm(apply_mcphase_theta, m, params={"theta": np.pi/2}), return_ptm(apply_mcphase_theta, m_prime, params={"theta": np.pi/2}))

2. $\mathcal{R}_Z(-\pi/2) \otimes \mathcal{R}_Z(-\pi/2)$

In [22]:
deco_dict["2"] = {}
deco_dict["2"]["q"] = 1/2
deco_dict["2"]["ptm"]  = np.kron(return_ptm(apply_mcphase_theta, m, params={"theta": -np.pi/2}), return_ptm(apply_mcphase_theta, m_prime, params={"theta": -np.pi/2}))

3. $\mathcal{\overline{E}}_Z \otimes \mathcal{I}$

In [ ]:
def apply_measure_and_prepare_z(op: np.ndarray, params=None):
    if op.shape[1] != 2:
        raise ValueError("Matrix shape error: the matrix should be 2x2")
    rho_0 = np.array([[1, 0], [0, 0]])
    rho_1 = np.array([[0, 0], [0, 1]])
    return rho_0*np.trace(rho_0@op) - rho_1*np.trace(rho_1@op)
    

In [24]:
deco_dict["3"] = {}
deco_dict["3"]["q"] = 1/2
deco_dict["3"]["ptm"] = np.kron(return_ptm(apply_measure_and_prepare_z, m, params=None), return_ptm(apply_mcphase_theta, m_prime, params={"theta": 2*np.pi}))

4. $\mathcal{\overline{E}}_Z\otimes \mathcal{R}_Z(\pi)$

In [25]:
deco_dict["4"] = {} 
deco_dict["4"]["q"] = -1/2
deco_dict["4"]["ptm"] = np.kron(return_ptm(apply_measure_and_prepare_z, m, params=None), return_ptm(apply_mcphase_theta, m_prime, params={"theta": np.pi}))

5. $\mathcal{I} \otimes \mathcal{\overline{E}}_Z$

In [26]:
deco_dict["5"] = {}
deco_dict["5"]["q"] = 1/2
deco_dict["5"]["ptm"] = np.kron(return_ptm(apply_mcphase_theta, m, params={"theta": 2*np.pi}), return_ptm(apply_measure_and_prepare_z, m, params=None))

6. $\mathcal{R}_Z(\pi)\otimes \mathcal{\overline{E}}_Z$

In [27]:
deco_dict["6"] = {}
deco_dict["6"]["q"] = -1/2
deco_dict["6"]["ptm"] =  np.kron(return_ptm(apply_mcphase_theta, m, params={"theta": np.pi}), return_ptm(apply_measure_and_prepare_z, m, params=None))

In [28]:
cut = sum([deco_dict[key]["q"]*deco_dict[key]["ptm"] for key in deco_dict.keys()])

In [29]:
np.max(np.abs(target_ptm_cz - cut))

np.float64(4.440892098500626e-16)

which shows that the decomposition is correct!